In [1]:
import time
import numpy as np
import sys,os
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from hfnet.settings import EXPER_PATH
from notebooks.utils import plot_images, plot_matches, add_frame

import tensorflow as tf
from tensorflow.python.saved_model import tag_constants
import logging
from tqdm import tqdm
import numpy as np
from scipy.spatial.transform import Rotation as R, Slerp 
from numpy.linalg import inv, norm
logging.basicConfig(format='[%(asctime)s %(levelname)s] %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
from hfnet.datasets import get_dataset  # noqa: E402
from hfnet.evaluation.loaders import export_loader  # noqa: E402
from hfnet.settings import EXPER_PATH  # noqa: E402
from scipy import spatial

/home/pang/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pang/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pang/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pang/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWa

In [2]:
tf.contrib.resampler  # import C++ op
class HFNet:
    def __init__(self, model_path, outputs):
        self.session = tf.Session()
        self.image_ph = tf.placeholder(tf.float32, shape=(None, None, 3))

        net_input = tf.image.rgb_to_grayscale(self.image_ph[None])
        tf.saved_model.loader.load(
            self.session, [tag_constants.SERVING], str(model_path),
            clear_devices=True,
            input_map={'image:0': net_input})

        graph = tf.get_default_graph()
        self.outputs = {n: graph.get_tensor_by_name(n+':0')[0] for n in outputs}
        self.nms_radius_op = graph.get_tensor_by_name('pred/simple_nms/radius:0')
        self.num_keypoints_op = graph.get_tensor_by_name('pred/top_k_keypoints/k:0')
    def inference(self, image, nms_radius=4, num_keypoints=1000):
        inputs = {
            self.image_ph: image[..., ::-1].astype(np.float),
            self.nms_radius_op: nms_radius,
            self.num_keypoints_op: num_keypoints,
        }
        return self.session.run(self.outputs, feed_dict=inputs)

model_path = Path(EXPER_PATH, 'saved_models/hfnet')
outputs = ['global_descriptor', 'keypoints', 'local_descriptors','scores']
hfnet = HFNet(model_path, outputs)

[10/24/2019 12:19:59 INFO] Restoring parameters from /home/pang/software/hfnet_venv/hfnet_exp/saved_models/hfnet/variables/variables


In [3]:
def compute_distance(desc1, desc2):
    # For normalized descriptors, computing the distance is a simple matrix multiplication.
    return 2 * (1 - desc1 @ desc2.T)

def match_with_ratio_test(desc1, desc2, thresh):
    dist = compute_distance(desc1, desc2)
    nearest = np.argpartition(dist, 2, axis=-1)[:, :2]
    dist_nearest = np.take_along_axis(dist, nearest, axis=-1)
    valid_mask = dist_nearest[:, 0] <= (thresh**2)*dist_nearest[:, 1]
    matches = np.stack([np.where(valid_mask)[0], nearest[valid_mask][:, 0]], 1)
    return matches

def undistortion(distP,fx,fy,cx,cy,w):
    u_p = [0.,0.]
    d_p = [0.,0.]
#     print("dp0:{} dp1:{} fx:{}".format(d_p[0],d_p[1],fx)
    d_p[0] = (float(distP[0]) - cx)/fx
    d_p[1] = (float(distP[1]) - cy)/fy
#     print("dp0:{} dp1:{} fx:{}".format((float(d_p[0]) - cx)/fx,float(d_p[1]) - cx,fx))
    mul2tanwby2 = np.tan(w / 2.0) * 2.0
    
#     Calculate distance from point to center.
    r_d = np.sqrt(d_p[0]*d_p[0] + d_p[1]*d_p[1])
    if mul2tanwby2 == 0 or r_d == 0:
        print("tanw error")
        return u_p
    
#     Calculate undistorted radius of point.
    kMaxValidAngle = 89.0;
    if abs(r_d * w) <= kMaxValidAngle:
        r_u = np.tan(r_d * w) / (r_d * mul2tanwby2)
    else:
        print('angle not valid')
        return u_p

    u_p[0] = d_p[0] * r_u;
    u_p[1] = d_p[1] * r_u;

    u_p[0] = u_p[0]* fx + cx;
    u_p[1] = u_p[1]* fy + cy;
    return u_p

def kpData2Dic(data):
    assert len(data)%3==0
    dict={}
    for i in range(0,len(data),3):
        dict[int(i/3)] = [float(data[i]),float(data[i+1]),int(data[i+2])]
    return dict
def loadSfmModel(filePath):
    p = open(filePath+'/points3D.txt')
    tmp3d = [ line.split(' ') for line in p.readlines()[3:]]
    points3D={}
    for line in tmp3d:
        e = [float(e) for e in line]
        points3D[e[0]] = e[1:]
    i=open(filePath+'/images.txt')
    images={}
    imagesData = i.readlines()[4:]
    for i in range(0,len(imagesData),2):
        l1 = imagesData[i].split(' ')
        l2 = imagesData[i+1].split(' ')
        if len(l1) == 10:
            kpDic = kpData2Dic(l2)
            name = l1[-1][:-1]
            pose = [float(e) for e in l1[1:8]]
            images[name]= {'pose':pose,'keypointsDic':kpDic}
    return points3D,images 

In [4]:
%load_ext autoreload
%autoreload 2
db_path =  "/home/pang/colmap_ws/office/office-1-5/sfm1/"
query_path ="/home/pang/data/dataset/iros2019slam/office/office-1-1/fisheye1/" 

db_list = os.listdir(db_path)
query_list = os.listdir(query_path)
query_list.sort()
db_list.sort()

db_image = lambda n: cv2.imread(db_path + n)[:, :, ::-1]
query_image = lambda n: cv2.imread(query_path + n)[:, :, ::-1]
images_db = [db_image(frame) for frame in db_list]
images_query = [query_image(frame) for frame in query_list]

db = [hfnet.inference(i) for i in images_db]
global_index = np.stack([d['global_descriptor'] for d in db])

In [15]:
modelPath = '/home/pang/colmap_ws/practice/hfnet/sparse/'
result_path = "/home/pang/colmap_ws/result_image1-1/"
w =  open("/home/pang/colmap_ws/result1-1/images.txt",'w')
w1 = open("/home/pang/colmap_ws/result1-1/reloc1-1.txt",'w')

if not os.path.isdir(result_path):
    os.makedirs(result_path)
else:
    filelist = os.listdir(result_path)
    for img in filelist:
        os.remove("{}/{}".format(result_path,img))   
        
plotMatch = False
dbPoints3D,dbImages = loadSfmModel(modelPath)
# fisheye
fx=284.981
fy=286.102
cx=425.244
cy=398.468
k1=-7.3047108016908169e-03
k2=4.3499931693077087e-02
k3=-4.1283041238784790e-02
k4=7.6524601317942142e-03

#284.981,286.102,425.244,398.468,-7.3047108016908169e-03,4.3499931693077087e-02,-4.1283041238784790e-02,7.6524601317942142e-03

count=0
K = np.array([[fx,0,cx],[0,fy,cy],[0,0,1]])
D = np.array([k1,k2,k3,k4])

### knn candidate
nn = 20
ratio_test = 0.7

relocResult=[]
locResultStr=[]
contestResultStr=[]
keypoints2D={}
# for i in range(len(images_query)):
    
for i in range(0,50,4):
    globalKpNum = []
    query = hfnet.inference(images_query[i])
    nearest = np.argsort(compute_distance(query['global_descriptor'], global_index))[:nn]
    undistKps=np.array([[0.,0.]],dtype=np.float32)
    imps=np.array([[0.,0.,0.]])
    allMatches=np.array([[0, 0]])

    for item in nearest:
        matches = match_with_ratio_test(query['local_descriptors'],
                                db[item]['local_descriptors'], ratio_test)
        if len(matches) <=3:
            continue
        distKp = query['keypoints'][matches[:,0]]
        distKp = np.array([distKp],dtype=np.float32)
      
        undistKp = cv2.undistortPoints(distKp,K,D,P=K)
        undistKp = undistKp.reshape((-1,2))
        
        if db_list[item] in dbImages:
            impsIdx = [dbImages[db_list[item]]['keypointsDic'][m[1]][2] for m in matches]
        else:
            continue
        impsIdx = np.array(impsIdx)
        a = np.where(np.array(impsIdx)>=0)  
        if len(a) <1:
            continue
        undistKp = undistKp[a]
        impsIdx=impsIdx[a]
        matches = matches[a]

        imp = [dbPoints3D[idx][:3] for idx in impsIdx]
        imp = np.array(imp)
        

        if len(imp) <1:
            continue;
        imps = np.concatenate((imps, imp),axis=0)
        undistKps = np.concatenate((undistKps,undistKp),axis = 0)
        allMatches = np.concatenate((allMatches,matches), axis = 0)
        globalKpNum.append([item,len(imps)-1])
    
   
    imps=imps[1:]
    undistKps=undistKps[1:]
    allMatches=allMatches[1:]


    if len(imps) <=4 :
        continue
    success, R_vec, t, inliers = cv2.solvePnPRansac(
        imps, undistKps, K, np.array([0., 0., 0., 0.]),
        iterationsCount=5000, reprojectionError=5.0,
        flags=cv2.SOLVEPNP_P3P)
    #check for unique elements number
    uniqueKps = undistKps[inliers]
    uniqueDic = {}
    for ui in uniqueKps:
        if not ui[0][0] in uniqueDic:
            uniqueDic[ui[0][0]] = ui
   
    if len(uniqueDic) <=10:
        continue
    if success and len(inliers) >=10 :
        inliers = inliers[:, 0]
        num_inliers = len(inliers)
#         inlier_ratio = len(inliers) / len(undistKp)
        success &= num_inliers >= 5

        ret, R_vec, t = cv2.solvePnP(
                imps[inliers], undistKps[inliers], K,
                np.array([0., 0., 0., 0.]), rvec=R_vec, tvec=t,
                useExtrinsicGuess=True, flags=cv2.SOLVEPNP_ITERATIVE)
        assert ret

        T_query_w = np.eye(4)
        T_query_w[:3, :3] = cv2.Rodrigues(R_vec)[0]
        T_query_w[:3, 3] = t[:, 0]
        q = (R.from_dcm(T_query_w[:3,:3])).as_quat()
        t = T_query_w[:3,3]
        ## for iros workshop
        T_w_query= np.linalg.inv(T_query_w)
        q1 = (R.from_dcm(T_w_query[:3,:3])).as_quat()
        t1 = T_w_query[:3,3]
        

        locResultStr.append("{} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f} 1 {}\n \n"
                        .format(count,q[3],q[0],q[1],q[2],t[0],t[1],t[2],query_list[i]))
        contestResultStr.append("{:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {:.6f} {}\n"
                        .format(q1[0],q1[1],q1[2],q1[3],t1[0],t1[1],t1[2],query_list[i]))
        matches = allMatches[inliers]

        if plotMatch == True:
            for j in range(len(allMatches)):
                if j in inliers:
                    continue
                else:
                    allMatches[j][1] = -1  
            for ii in range(len(globalKpNum)):

                if ii==0:
                    m = allMatches[:globalKpNum[ii][1]]
                    m = m[m[:,1] >= 0]
                    plot_matches(images_query[i], query['keypoints'],
                     images_db[globalKpNum[ii][0]], db[globalKpNum[ii][0]]['keypoints'],
                     m, color=(0, 1, 0), dpi=100,thickness=0.5,kp_size=6)
                else:
                    m = allMatches[globalKpNum[ii-1][1]:globalKpNum[ii][1]]
                    m = m[m[:,1] >= 0]
                    plot_matches(images_query[i], query['keypoints'],
                     images_db[globalKpNum[ii][0]], db[globalKpNum[ii][0]]['keypoints'],
                     m, color=(0, 1, 0), dpi=100,thickness=0.5,kp_size=6)
                plt.savefig(result_path+str(count)+"_matches"+str(ii)+ ".jpg")
                plt.cla()
                plt.close('all')
                
            allMatches=allMatches[inliers]
            if len(nearest) > 1:
                plot_images([images_query[i]], keypoints=[query['keypoints'][allMatches[:,0]]])
                plt.savefig(result_path+str(count)+"_matches.jpg")
                plt.cla()
                plt.close('all')              
            
    count+=1
    print("relocalization happend with:{} total reloc number:{}".format(query_list[i],count))
    
    
    
    
    
        



          
          
for l in locResultStr:
    w.write(l)
w.close()  

for l in contestResultStr:
    w1.write(l)
w1.close() 

    

relocalization happend with:1560000083949067.png total reloc number:1
relocalization happend with:1560000084082392.png total reloc number:2
relocalization happend with:1560000084215726.png total reloc number:3
relocalization happend with:1560000084349059.png total reloc number:4
relocalization happend with:1560000084482393.png total reloc number:5
relocalization happend with:1560000084615691.png total reloc number:6
relocalization happend with:1560000084749025.png total reloc number:7
relocalization happend with:1560000084882359.png total reloc number:8
relocalization happend with:1560000085015692.png total reloc number:9
relocalization happend with:1560000085149003.png total reloc number:10
relocalization happend with:1560000085282336.png total reloc number:11
relocalization happend with:1560000085415670.png total reloc number:12
relocalization happend with:1560000085549004.png total reloc number:13


In [6]:
print(query_list)

['1560000083949067.png', '1560000083982400.png', '1560000084015734.png', '1560000084049067.png', '1560000084082392.png', '1560000084115725.png', '1560000084149059.png', '1560000084182392.png', '1560000084215726.png', '1560000084249059.png', '1560000084282392.png', '1560000084315726.png', '1560000084349059.png', '1560000084382393.png', '1560000084415726.png', '1560000084449059.png', '1560000084482393.png', '1560000084515726.png', '1560000084549060.png', '1560000084582358.png', '1560000084615691.png', '1560000084649025.png', '1560000084682358.png', '1560000084715691.png', '1560000084749025.png', '1560000084782358.png', '1560000084815692.png', '1560000084849025.png', '1560000084882359.png', '1560000084915692.png', '1560000084949025.png', '1560000084982359.png', '1560000085015692.png', '1560000085049026.png', '1560000085082336.png', '1560000085115669.png', '1560000085149003.png', '1560000085182336.png', '1560000085215670.png', '1560000085249003.png', '1560000085282336.png', '15600000853156